In [0]:
dbutils.fs.put("/FileStore/tables/sample.json","""{
  "name":"MSFT","location":"Redmond", "satellites": ["Bay Area", "Shanghai"],
  "goods": {
    "trade":true, "customers":["government", "distributer", "retail"],
    "orders":[
        {"orderId":1,"orderTotal":123.34,"shipped":{"orderItems":[{"itemName":"Laptop","itemQty":20},{"itemName":"Charger","itemQty":2}]}},
        {"orderId":2,"orderTotal":323.34,"shipped":{"orderItems":[{"itemName":"Mice","itemQty":2},{"itemName":"Keyboard","itemQty":1}]}}
    ]}}
{"name":"Company1","location":"Seattle", "satellites": ["New York"],
  "goods":{"trade":false, "customers":["store1", "store2"],
  "orders":[
      {"orderId":4,"orderTotal":123.34,"shipped":{"orderItems":[{"itemName":"Laptop","itemQty":20},{"itemName":"Charger","itemQty":3}]}},
      {"orderId":5,"orderTotal":343.24,"shipped":{"orderItems":[{"itemName":"Chair","itemQty":4},{"itemName":"Lamp","itemQty":2}]}}
    ]}}
{"name": "Company2", "location": "Bellevue",
  "goods": {"trade": true, "customers":["Bank"], "orders": [{"orderId": 4, "orderTotal": 123.34}]}}
{"name": "Company3", "location": "Kirkland"}""",True)

Wrote 1073 bytes.
Out[50]: True

In [0]:
df_json = spark.read.option("multiLine","true").json("/FileStore/tables/sample.json")

In [0]:
display(df_json)

goods,location,name,satellites
"List(List(government, distributer, retail), List(List(1, 123.34, List(List(List(Laptop, 20), List(Charger, 2)))), List(2, 323.34, List(List(List(Mice, 2), List(Keyboard, 1))))), true)",Redmond,MSFT,"List(Bay Area, Shanghai)"


In [0]:
df_json.printSchema()

root
 |-- goods: struct (nullable = true)
 |    |-- customers: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- orders: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- orderId: long (nullable = true)
 |    |    |    |-- orderTotal: double (nullable = true)
 |    |    |    |-- shipped: struct (nullable = true)
 |    |    |    |    |-- orderItems: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |-- itemName: string (nullable = true)
 |    |    |    |    |    |    |-- itemQty: long (nullable = true)
 |    |-- trade: boolean (nullable = true)
 |-- location: string (nullable = true)
 |-- name: string (nullable = true)
 |-- satellites: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [0]:
def child_struct(nested_df):
    # Creating python list to store dataframe metadata
    list_schema = [((), nested_df)]
    # Creating empty python list for final flattern columns
    flat_columns = []

    while len(list_schema) > 0:
      # Removing latest or recently added item (dataframe schema) and returning into df variable  
          parents, df = list_schema.pop()
          flat_cols = [  col(".".join(parents + (c[0],))).alias("_".join(parents + (c[0],))) for c in df.dtypes if c[1][:6] != "struct"   ]
      
          struct_cols = [  c[0]   for c in df.dtypes if c[1][:6] == "struct"   ]
      
          flat_columns.extend(flat_cols)
          #Reading  nested columns and appending into stack list
          for i in struct_cols:
                projected_df = df.select(i + ".*")
                list_schema.append((parents + (i,), projected_df))
    return nested_df.select(flat_columns)

In [0]:
from pyspark.sql.functions import *
def master_array(df):
    array_cols = [c[0] for c in df.dtypes if c[1][:5]=="array"]
    while len(array_cols)>0:
        for c in array_cols:
            df = df.withColumn(c,explode_outer(c))
        df = child_struct(df)
        array_cols = [c[0] for c in df.dtypes if c[1][:5]=="array"]
    return df

In [0]:
df_output = master_array(df_json)

In [0]:
display(df_output)

location,name,satellites,goods_customers,goods_trade,goods_orders_orderId,goods_orders_orderTotal,goods_orders_shipped_orderItems_itemName,goods_orders_shipped_orderItems_itemQty
Redmond,MSFT,Bay Area,government,true,1,123.34,Laptop,20
Redmond,MSFT,Bay Area,government,true,1,123.34,Charger,2
Redmond,MSFT,Bay Area,government,true,2,323.34,Mice,2
Redmond,MSFT,Bay Area,government,true,2,323.34,Keyboard,1
Redmond,MSFT,Bay Area,distributer,true,1,123.34,Laptop,20
Redmond,MSFT,Bay Area,distributer,true,1,123.34,Charger,2
Redmond,MSFT,Bay Area,distributer,true,2,323.34,Mice,2
Redmond,MSFT,Bay Area,distributer,true,2,323.34,Keyboard,1
Redmond,MSFT,Bay Area,retail,true,1,123.34,Laptop,20
Redmond,MSFT,Bay Area,retail,true,1,123.34,Charger,2
